In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import subprocess
import sys
import io
import gradio as gr
import numpy as np
import random
import spaces
import torch
from diffusers import Flux2Pipeline, Flux2Transformer2DModel
from diffusers import BitsAndBytesConfig as DiffBitsAndBytesConfig
import requests
from PIL import Image
import json
import base64
from dotenv import load_dotenv
from huggingface_hub import login, InferenceClient
from helper_dev_utils import get_auto_logger
logger = get_auto_logger()

In [ ]:
# Hugging Face 로그인 (FLUX 모델 접근 권한 필요)
load_dotenv()

# 환경 변수에서 토큰 가져오기 또는 직접 입력
hf_token = os.getenv("HF_TOKEN")
logger.info(f"Hugging Face Token: {hf_token[:2]} ... {hf_token[-2:]}")

if hf_token:
    login(token=hf_token)
    print("✓ Logged in with HF_TOKEN from environment")
else:
    # 수동 로그인 (토큰 입력 필요)
    login()
    print("✓ Manual login completed")

In [ ]:
import sys
import gc
import torch
from pathlib import Path
from PIL import Image
from typing import Optional
import matplotlib.pyplot as plt

# [수정 1] AutoPipelineForImage2Image 추가 import
from diffusers import DiffusionPipeline, AutoPipelineForImage2Image 
from diffusers.utils import logging as diffusers_logging

# 불필요한 경고 숨기기 (선택사항)
diffusers_logging.set_verbosity_error()

# 프로젝트 경로 설정 (기존 유지)
project_root = Path.cwd().parent.parent / "src" / "nanoCocoa_aiserver"
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

try:
    from config import TORCH_DTYPE
    from services.monitor import flush_gpu
    from helper_dev_utils import get_auto_logger
except ImportError:
    TORCH_DTYPE = torch.bfloat16
    def flush_gpu(): pass
    def get_auto_logger():
        import logging
        return logging.getLogger("FluxGenerator")

logger = get_auto_logger()

class FluxGenerator:
    def __init__(self):
        self.model_id = "diffusers/FLUX.2-dev-bnb-4bit"
        self.pipe_txt2img = None
        self.pipe_img2img = None
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        logger.info(f"FluxGenerator initialized using: {self.model_id}")

    def _flush_gpu(self):
        flush_gpu()
        gc.collect()
        if torch.cuda.is_available():
            torch.cuda.empty_cache()

    def _load_pipelines(self):
        if self.pipe_txt2img is not None:
            return

        logger.info(f"Loading FLUX.2 Pipeline...")
        self._flush_gpu()
        
        try:
            # 1. Txt2Img 파이프라인 로드
            from transformers import BitsAndBytesConfig
            quantization_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_compute_dtype=torch.bfloat16,
                bnb_4bit_quant_type="nf4",
            )
            max_memory = {0: "20GB", "cpu": "80GB"}

            self.pipe_txt2img = DiffusionPipeline.from_pretrained(
                self.model_id,
                torch_dtype=TORCH_DTYPE,
                device_map="balanced",
                max_memory=max_memory,
                quantization_config=quantization_config
            )

            # CPU Offload (VRAM 절약)
            if not hasattr(self.pipe_txt2img, "hf_device_map"):
                 self.pipe_txt2img.enable_model_cpu_offload()

            # VAE Tiling (OOM 방지)
            if hasattr(self.pipe_txt2img, "vae"):
                self.pipe_txt2img.vae.enable_tiling()
                self.pipe_txt2img.vae.enable_slicing()

            # [핵심 수정] AutoPipeline을 사용하여 Img2Img 파이프라인 자동 변환
            # Flux 1인지 2인지 상관없이 적절한 클래스로 매핑해줍니다.
            try:
                self.pipe_img2img = AutoPipelineForImage2Image.from_pipe(self.pipe_txt2img)
                logger.info("Img2Img Pipeline created via AutoPipeline.")
            except Exception as e:
                logger.error(f"Failed to create Img2Img pipe: {e}")
                # 정말 실패했을 경우엔 img2img를 None으로 두어 폴백 처리
                self.pipe_img2img = None

            logger.info("FLUX.2 Pipelines loaded successfully")
            
        except Exception as e:
            logger.error(f"Failed to load pipeline: {e}")
            self._flush_gpu()
            raise e

    def generate_background_and_compose_image(
        self,
        prompt: str,
        input_image: Optional[Image.Image] = None,
        width: int = 1024,
        height: int = 1024,
        num_inference_steps: int = 30,
        guidance_scale: float = 3.5,
        strength: float = 0.75,
        seed: int = None,
        **kwargs
    ) -> Image.Image:
        
        self._load_pipelines()
        self._flush_gpu()
        
        generator = torch.Generator(device="cpu").manual_seed(seed) if seed else None

        common_args = {
            "prompt": prompt,
            "num_inference_steps": num_inference_steps,
            "guidance_scale": guidance_scale,
            "generator": generator,
            "max_sequence_length": 512,
            "width": width,
            "height": height
        }

        try:
            with torch.no_grad():
                if input_image is None:
                    # [Txt2Img]
                    logger.info("Generating [Txt2Img]...")
                    result = self.pipe_txt2img(**common_args)
                else:
                    # [Img2Img]
                    logger.info("Generating [Img2Img]...")
                    
                    # 1. Img2Img 파이프라인이 정상적으로 로드되었는지 확인
                    if self.pipe_img2img is None:
                        raise RuntimeError("Img2Img pipeline failed to load. Cannot perform image-to-image generation.")

                    # 2. 이미지 전처리
                    if input_image.size != (width, height):
                        input_image = input_image.resize((width, height), Image.LANCZOS)
                    
                    common_args["image"] = input_image
                    common_args["strength"] = strength
                    
                    result = self.pipe_img2img(**common_args)
            
            return result.images[0]

        except Exception as e:
            logger.error(f"Error during generation: {e}")
            raise e
        finally:
            self._flush_gpu()

# -------------------------------------------------------------------------
# 실행 테스트
# -------------------------------------------------------------------------
if __name__ == "__main__":
    if 'flux_generator' in globals():
        del flux_generator
    gc.collect()
    torch.cuda.empty_cache()
    
    flux_generator = FluxGenerator()
    
    # 이미지 생성 테스트
    ad_prompt = "High-quality product photography, rustic wooden table, soft sunlight, bokeh background, 8k"
    
    # 예시 이미지가 없다면 새로 생성 (테스트용)
    if not Path("image01.png").exists():
        Image.new('RGB', (1024, 1024), color='white').save("image01.png")
    
    image01 = Image.open("image01.png")
    
    try:
        ad_image = flux_generator.generate_background_and_compose_image(
            prompt=ad_prompt,
            input_image=image01,
            width=1024,
            height=1024,
            strength=0.75
        )
        
        plt.figure(figsize=(4,4))
        plt.imshow(ad_image)
        plt.axis('off')
        plt.show()
    except Exception as e:
        print(f"Execution Failed: {e}")

In [ ]:
# 메모리 정리
flux_generator.unload()
logger.info("All resources cleaned up")